In [1]:
import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words = sorted(stop_words)

def extractSentence(record):
    try:
        split_record = record.strip().split('\t')
        genre = split_record[3]
        sentences = []
        if (len(split_record[8])>0):
            sentences.append(split_record[8])
        if (len(split_record[9])>0):
            sentences.append(split_record[9])
        if(len(sentences)>0):
            return [(genre,sentence) for sentence in sentences]
        else:
            return ()
    except:
        return()


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1589981439126_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
train = sc.textFile('s3://comp5349-bgao6342/Assignment/train.tsv', 16)

# Extract and remove the header
header = train.first()
data_noheader = train.filter(lambda x: x != header) 
sentence_genre = data_noheader.flatMap(extractSentence)

# convert RDD to dataframe
schema = StructType([StructField('genre', StringType(), True),
                     StructField('sentence', StringType(), True)])
sentence_df = sqlContext.createDataFrame(sentence_genre, schema)

# TFIDF encoding 
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

# extract words from sentence
regexTokenizer = RegexTokenizer(inputCol='sentence', outputCol= 'words', gaps=False, pattern="\\w+") 
wordsData = regexTokenizer.transform(sentence_df) 

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures",numFeatures=2000)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)  
feature = rescaledData.select("genre", "features")

# clustering 
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Trains a k-means model
kmeans = KMeans(featuresCol="features", predictionCol="prediction").setK(5).setSeed(5)
model = kmeans.fit(feature)

# Make predictions
predictions = model.transform(feature)
prediction_genre = predictions.select("prediction","genre")
prediction_count = prediction_genre.groupBy("genre","prediction").count().orderBy("genre","prediction")
prediction_count.show(25)



   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+------+
|     genre|prediction| count|
+----------+----------+------+
|   fiction|         0|143591|
|   fiction|         1|  9236|
|   fiction|         2|  1292|
|   fiction|         4|   577|
|government|         0|104137|
|government|         1| 47476|
|government|         2|   263|
|government|         4|  2824|
|     slate|         0|123182|
|     slate|         1| 29594|
|     slate|         2|  1170|
|     slate|         3|     3|
|     slate|         4|   663|
| telephone|         0|118294|
| telephone|         1|  4150|
| telephone|         2| 37688|
| telephone|         3|  5958|
| telephone|         4|   606|
|    travel|         0|106035|
|    travel|         1| 48194|
|    travel|         2|    59|
|    travel|         4|   412|
+----------+----------+------+

In [4]:
# convert datagrame to a list of tuples 
prediction_list = [(row['prediction']) for row in prediction_count.collect()]
genre_list = [(row['genre']) for row in prediction_count.collect()]
count_list = [(row['count']) for row in prediction_count.collect()]
genre_prediction_count = [] 

for i in range(len(prediction_list)):
    temp = (genre_list[i],prediction_list[i],count_list[i])
    genre_prediction_count.append(temp)

fiction_sum= 0
governement_sum = 0
slate_sum = 0
tele_sum= 0
travel_sum= 0

# calculate the percentage 
for i in range(len(genre_prediction_count)):
    genre = genre_prediction_count[i][0]
    count = genre_prediction_count[i][2]
    if(genre == 'fiction'):
        fiction_sum +=count
    if(genre == 'government'):
        governement_sum +=count
    if(genre == 'slate'):
        slate_sum +=count
    if(genre == 'telephone'):
        tele_sum +=count
    if(genre == 'travel'):
        travel_sum +=count

genre_prediction_percent = []
for i in range(len(genre_prediction_count)):
    genre = genre_prediction_count[i][0]
    if(genre == 'fiction'):
        percentage = round(genre_prediction_count[i][2]/fiction_sum*100,2)
        genre_prediction_percent.append((genre_prediction_count[i][0],genre_prediction_count[i][1],percentage))
    if(genre == 'government'):
        percentage = round(genre_prediction_count[i][2]/governement_sum*100,2)
        genre_prediction_percent.append((genre_prediction_count[i][0],genre_prediction_count[i][1],percentage))
    if(genre == 'slate'):
        percentage = round(genre_prediction_count[i][2]/slate_sum*100,2)
        genre_prediction_percent.append((genre_prediction_count[i][0],genre_prediction_count[i][1],percentage))
    if(genre == 'telephone'):
        percentage = round(genre_prediction_count[i][2]/tele_sum*100,2)
        genre_prediction_percent.append((genre_prediction_count[i][0],genre_prediction_count[i][1],percentage))
    if(genre == 'travel'):
        percentage = round(genre_prediction_count[i][2]/travel_sum*100,2)
        genre_prediction_percent.append((genre_prediction_count[i][0],genre_prediction_count[i][1],percentage))
    
print(genre_prediction_percent)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('fiction', 0, 92.82), ('fiction', 1, 5.97), ('fiction', 2, 0.84), ('fiction', 4, 0.37), ('government', 0, 67.32), ('government', 1, 30.69), ('government', 2, 0.17), ('government', 4, 1.83), ('slate', 0, 79.67), ('slate', 1, 19.14), ('slate', 2, 0.76), ('slate', 3, 0.0), ('slate', 4, 0.43), ('telephone', 0, 70.96), ('telephone', 1, 2.49), ('telephone', 2, 22.61), ('telephone', 3, 3.57), ('telephone', 4, 0.36), ('travel', 0, 68.54), ('travel', 1, 31.15), ('travel', 2, 0.04), ('travel', 4, 0.27)]